In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing necessary packages

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

# Import Training Dataset

In [ ]:
df_train = pd.read_csv("/kaggle/input/credit-score-classification/train.csv")
df_train.head()

# Import Testing Dataset

In [ ]:
df_test = pd.read_csv("/kaggle/input/credit-score-classification/test.csv")
df_test.head()

# Data Understanding and Brief EDA

In [ ]:
df_train.info()

As we can see some of the columns have null entries so we have to deal with that. Also, we can check the data type of each columnn and correct if necessary 

In [ ]:
df_train.shape

In [ ]:
df_train.describe()

In [ ]:
df_train.describe(include=object).T

In [ ]:
df = df_train.copy()
# replace with df_test for cleaning of test data.

# Data Cleaning

In [ ]:
# Check Duplicated Data
df.duplicated().any()

In [ ]:
df["Customer_ID"].isnull().sum()
# "Customer_ID" column is used for group mode/

In [ ]:
df["Customer_ID"].describe()

In [ ]:
#dropping unnecessary columns
df_1 = df.drop(columns=["ID","Name","SSN"])

In [ ]:
# Detect strange values (in those columns which have string)
# apart from the NaN Values and convert them to NaN.
object_col = df_1.describe(include='O').columns
object_col

In [ ]:
for i in object_col:
    print(f"----------------{i}------------------")
    print(df_1[i].value_counts())
    print("\n")

In [ ]:
# Clear strange values apart from the NaN Values
def text_cleaning(data):
    if data is np.NaN or not isinstance(data, str):
        return data
    else:
        return str(data).strip("_ ,")

In [ ]:

# apply function is not working here.
# use apply for working with one column
# use apply map for whole data
# https://www.geeksforgeeks.org/difference-between-map-applymap-and-apply-methods-in-pandas/


In [ ]:
df_1 = df_1.applymap(text_cleaning)
df_1 = df_1.replace(["", "nan", "!@9#%8"],np.NaN)
# "!@9#%8" in payment behiaviou

In [ ]:
df_1["Occupation"][0:10]
# see 8th value to check if the above code worked or not.

In [ ]:
def missing_data(dataframe):
    dataframe_copy = dataframe.copy()
    dataframe_copy = dataframe_copy.isnull().sum().reset_index()
    dataframe_copy.columns = ["Feature", "Null_Values"]
    total_data = dataframe.shape[0]
#     print(total_data)
    dataframe_copy["%_age"] = dataframe_copy["Null_Values"] / total_data * 100
    return dataframe_copy[dataframe_copy["Null_Values"] != 0]

In [ ]:
# Check for missing values
missing_data(df_1)

### No null values in the target column so we are safe. Other null value should be taken care off 

In [ ]:
df_1.info()

In [ ]:
print(df_1.describe(include='O').columns)


## Data type Fix
### columns that needs to be converted to int/float
1. Age
2. Annual_Income
3. Num_of_Loan
4. Num_of_Delayed_Payment
5. Changed_Credit_Limit
6. Outstanding_Debt
7. Amount_invested_monthly
8. Monthly_Balance

If we would have tried converting the data type befor cleaning than we would have experienced errors.

In [ ]:
df_1["Month"] = pd.to_datetime(df_1.Month, format='%B').dt.month

In [ ]:
lst1 = ["Age", "Annual_Income", "Num_of_Loan", "Num_of_Delayed_Payment",
        "Changed_Credit_Limit", "Outstanding_Debt", "Amount_invested_monthly",
        "Monthly_Balance"]
for i in lst1:
    df_1[i] = df_1[i].astype(float)

In [ ]:
df_1["Customer_ID"][0]

In [ ]:
int("0xd40",16)
# Hexadecimal to a Decimal integer.

In [ ]:
df_1['Customer_ID'] = df_1.Customer_ID.apply(lambda x: int(x[4:], 16))

In [ ]:
print(df_1.describe(include='O').columns)

STill we have some categprical variable to convert we will tackle them.

In [ ]:
df_1["Credit_History_Age"][1]

In [ ]:
pd.notnull(df_1["Credit_History_Age"][1])

In [ ]:
"22 Years and 1 Months".split(" ")

In [ ]:
# 22 Years and 1 Months
def month_convertor(x):
    if pd.notnull(x):
        num1 = int(x.split(" ")[0])
        num2 = int(x.split(" ")[3])
        return (num1 * 12) + num2
    else:
        return x

In [ ]:
df_1.groupby('Customer_ID')['Credit_History_Age'].apply(list)

Credit_History_Age

In [ ]:
df_1["Credit_History_Age"] = df_1["Credit_History_Age"].apply(month_convertor)

In [ ]:
df_1.groupby('Customer_ID')['Credit_History_Age'].apply(list)

TYpe of loan

In [ ]:
s = "Credit-Builder Loan, Home Equity Loan, Student Loan, and Debt Consolidation Loan"
s.strip().replace("and ","").replace(",","").split("Loan")

In [ ]:
d = ['Credit-Builder ', ' Home Equity ', ' Student ', ' Debt Consolidation ', '']
[i.strip(" ")+"Loan" for i in d if len(i)>0]
# [f(x) if condition else g(x) for x in sequence]
# [f(x) for x in sequence if condition]

In [ ]:
def type_of_loan(x):
    if pd.notna(x):
        x = x.lower().replace("and ", "")
        x = x.replace(", ", ",")
        x = x.strip()
        return x
    else:
        return x

In [ ]:
df_1["Type_of_Loan"] = df_1["Type_of_Loan"].apply(type_of_loan)

In [ ]:
df_1.groupby("Customer_ID")["Type_of_Loan"].value_counts(dropna=False)

In [ ]:
df_1.groupby("Customer_ID")["Type_of_Loan"].apply(list)

In [ ]:
len(df_1.groupby("Customer_ID")["Type_of_Loan"].apply(list)[1006])

In [ ]:
unique_loans = []
for each_row in df_1["Type_of_Loan"].dropna():
#     print(each_row.split(","))
    for each_loan in each_row.split(","):
        if each_loan not in unique_loans:
            unique_loans.append(each_loan)

In [ ]:
unique_loans = dict(enumerate(sorted(unique_loans)))
unique_loans

## Object column : Reassign Group Mode Values to null values

In [ ]:
missing_data(df_1)

In [ ]:
temp = pd.DataFrame({"A":[None, "4"],"B":[54,54]})
temp

In [ ]:
for columns in temp.columns:
    if temp[columns].isin([None]).sum():
        print(columns)

In [ ]:
df_1["Occupation"].value_counts(dropna=False)

In [ ]:
df_1["Occupation"].mode()[0]
# but we need group mode

In [ ]:
stats.mode(["Lay","Lay"])

Difference between transform and apply. check the data shape. its same in transform

In [ ]:
df_1.groupby("Customer_ID")["Occupation"].transform(lambda x : stats.mode(x)[0][0])

In [ ]:
df_1.groupby("Customer_ID")["Occupation"].apply(lambda x : stats.mode(x)[0][0])

In [ ]:
# for column with data type object.
def group_mode(dataframe, groupby, column):
    result = dataframe.groupby(groupby)[column].transform(lambda x : x.fillna(stats.mode(x)[0][0]))
    return result

Occupation

In [ ]:
#before:
df_1.groupby("Customer_ID")["Occupation"].apply(list)

In [ ]:
df_1["Occupation"] = group_mode(df_1, "Customer_ID", column = "Occupation")

In [ ]:
#after
df_1.groupby("Customer_ID")["Occupation"].apply(list)

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(data=df_1,x="Occupation")
plt.xticks(rotation=45)
plt.show()

TYpe of loan

In [ ]:
df_1["Type_of_Loan"].value_counts(dropna=False)

In [ ]:
#before
df_1.groupby("Customer_ID")["Type_of_Loan"].apply(list)

In [ ]:
# group mode is not required 
# since for a customer whose one instance has nan, its all instances have nan.
df_1["Type_of_Loan"].replace(np.NaN,"No Data", inplace=True)

In [ ]:
#after
df_1.groupby("Customer_ID")["Type_of_Loan"].apply(list)

credit mix

In [ ]:
# befor
df_1.groupby("Customer_ID")["Credit_Mix"].apply(list)

In [ ]:
# group mode is required
df_1["Credit_Mix"] = group_mode(df_1, "Customer_ID", column = "Credit_Mix")

In [ ]:
# after
df_1.groupby("Customer_ID")["Credit_Mix"].apply(list)

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(data=df_1,x="Credit_Mix")
plt.xticks(rotation=45)
plt.show()

Payment_Behaviour

In [ ]:
df_1["Payment_Behaviour"].value_counts(dropna=False)

In [ ]:
# before
df_1.groupby("Customer_ID")["Payment_Behaviour"].apply(list)
# check customer id = 1009

In [ ]:
df_1.groupby("Customer_ID")["Payment_Behaviour"].apply(list)[1009]

In [ ]:
df_1["Payment_Behaviour"] = group_mode(df_1, "Customer_ID", column = "Payment_Behaviour")

In [ ]:
#after
df_1.groupby("Customer_ID")["Payment_Behaviour"].apply(list)[1009]

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(data=df_1,x="Payment_Behaviour")
plt.xticks(rotation=45)
plt.show()

In [ ]:
missing_data(df_1[list(df_1.describe(include="O").columns)])
# NO object data type column has missing values

In [ ]:
missing_data(df_1[list(df_1.describe(exclude="O").columns)])
# NO object data type column has missing values

In [ ]:
list(df_1.describe(include="O").columns)

In [ ]:
# pd.DataFrame(df_1.groupby("Customer_ID")["Occupation"].apply(lambda x : stats.mode(x)[0][0])).reset_index()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(data = df_1, x = "Occupation")
plt.xticks(rotation=45)
plt.show()

In [ ]:
df_1["Type_of_Loan"].value_counts().index

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(data = df_1, x = "Credit_Mix")
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(data = df_1, x = "Payment_of_Min_Amount")
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(data = df_1, x = "Payment_Behaviour")
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(data = df_1, x = "Credit_Score")
plt.title("Class Distribution")
plt.show()

In [ ]:
# fig, ax = plt.subplots(3,3, figsize = (20,20))
# for i, j in zip(ax.ravel(), list(df_1.describe(include="O").columns)):
#     if len(list(df_1[j].value_counts().index)) < 20:
#         sns.countplot(data = df_1, x = j, ax = i)
# plt.show()

Now null values of numerical data type columns.


In [ ]:
# Define Outlier Range
def get_iqr_lower_upper(dataframe, column, multiply=1.5):
    q1 = dataframe[column].quantile(0.25)
    q3 = dataframe[column].quantile(0.75)
    iqr = q3 - q1
    
    lower = q1-iqr*multiply
    upper = q3+iqr*multiply
    affect = dataframe.loc[(dataframe[column]<lower)|(dataframe[column]>upper)].shape
    print('Outliers:', affect)
    return lower, upper

In [ ]:
get_iqr_lower_upper(df_1,"Monthly_Inhand_Salary")

In [ ]:
# # for column with data type numerical.
# def group_mean(df, groupby="Customer_ID",column=None):
# #     result = df.dropna().groupby(groupby)[column].transform(np.mean)
#     result = df.groupby(groupby)[column].transform(np.nanmean)
#     return result

In [ ]:
df_1.groupby("Customer_ID")["Monthly_Inhand_Salary"].apply(list)[50984]

In [ ]:
np.nanmean(df_1.groupby("Customer_ID")["Monthly_Inhand_Salary"].apply(list)[50984])

In [ ]:
df_1["Monthly_Inhand_Salary"].isnull()

In [ ]:
df_1["Monthly_Inhand_Salary"].notna()
# thats why notna() is used in the main function.

In [ ]:
df_1[df_1["Monthly_Inhand_Salary"].notna()].groupby("Customer_ID")["Monthly_Inhand_Salary"].apply(list)

In [ ]:
df_1[df_1["Monthly_Inhand_Salary"].notna()].groupby("Customer_ID")["Monthly_Inhand_Salary"].apply(list).apply(lambda x: stats.mode(x))
# stats mode is applied on the list of numbers.

In [ ]:
df_1[df_1["Monthly_Inhand_Salary"].notna()].groupby("Customer_ID")["Monthly_Inhand_Salary"].apply(list).apply(lambda x: stats.mode(x)[0][0])

In [ ]:
temp = df_1[df_1["Monthly_Inhand_Salary"].notna()].groupby("Customer_ID")["Monthly_Inhand_Salary"].apply(list).apply(lambda x: stats.mode(x)[0][0])
temp.describe()
# see its min and max which are same as applying min max on the temp

In [ ]:
temp.apply([min,max])

In [ ]:
t1, t2 = temp.apply([min,max])
t1

In [ ]:
df_1.groupby("Customer_ID")["Monthly_Inhand_Salary"].get_group(31744).mode()

In [ ]:
# def con_1(x):
#     print(x)
#     print(type(x))
#     print(x.mode())
    
    
#     if not x.mode().empty:
#         x = x.mode()[0]
#     else:
#         x = np.NaN

In [ ]:
# df_1.groupby("Customer_ID")["Monthly_Inhand_Salary"].transform(con_1)

In [ ]:
df_1.groupby("Customer_ID")["Monthly_Inhand_Salary"].transform(lambda x: x.mode()[0] if not x.mode().empty else np.NaN)

In [ ]:
def replace_group_outliers_fillna(dataframe, groupby, column):
    # drop nan values before doing anything
    df_dropped = dataframe[dataframe[column].notna()]
    # for each customer(group) : list all the values in the column for a group
    df_grouped = df_dropped.groupby(groupby)[column].apply(list)
    # find groupwise mode
    df_grouped = df_grouped.apply(lambda x: stats.mode(x)[0][0])
    # above dataframe will have groupby modes the calculate min and max of all customers
    mini, maxi = df_grouped.apply([min, max])
    # assign Wrong Values to NaN
    col = dataframe[column].apply(lambda x: np.NaN if ((x<mini)|(x>maxi)) else x) # or condition
    # ------------------------------fill with local mode
    # By default, missing values are not considered while calcuating mode
    # type(x) >> pandas series
    # before assinging check if mode is empty of not
    mode_by_group = dataframe.groupby(groupby)[column].transform(lambda x: x.mode()[0] if not x.mode().empty else np.NaN)
    result = col.fillna(mode_by_group)
    # mode_by_group will provide the values to be filled for nulls  
    # fillna takes the index of null value in col and search it in mode_by_group and when it finds it
    # it will replace nan with the found value.
    return result

In [ ]:
temp = pd.DataFrame([[np.nan, 2, np.nan, 0],
                   [3, 4, np.nan, 1],
                   [np.nan, np.nan, np.nan, np.nan],
                   [np.nan, 3, np.nan, 4]],
                  columns=list("ABCD"))
temp

In [ ]:
temp2 = pd.DataFrame([[1, 1, 1, 1],
                   [1, 1, 1, 1],
                   [1, 1, 1, 1],
                   [1, 1, 1, 1]],
                  columns=list("ABCD"))
temp2

In [ ]:
temp.fillna(temp2)

In [ ]:
missing_data(df_1)

In [ ]:
df_1.info()

AGe

In [ ]:
#Before
df_1.groupby("Customer_ID")["Age"].apply(list)

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Age")
plt.title("Box Plot of Age")
plt.show()

In [ ]:
df_1["Age"] = replace_group_outliers_fillna(df_1, "Customer_ID", "Age")

In [ ]:
#after
df_1.groupby("Customer_ID")["Age"].apply(list)

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Age")
plt.title("Box Plot of Age")
plt.show()

In [ ]:
get_iqr_lower_upper(df_1, "Age")
# No outliers

In [ ]:
plt.figure(figsize = (10,5))
sns.histplot(data = df_1, x = "Age",kde=True)
plt.show()

Anual Income

In [ ]:
#Before
df_1.groupby("Customer_ID")["Annual_Income"].apply(list)

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Annual_Income")
plt.title("Box Plot of Annual_Income")
plt.show()

In [ ]:
df_1["Annual_Income"] = replace_group_outliers_fillna(df_1, "Customer_ID", "Annual_Income")

In [ ]:
#after
df_1.groupby("Customer_ID")["Annual_Income"].apply(list)

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Annual_Income")
plt.title("Box Plot of Annual_Income")
plt.show()

In [ ]:
get_iqr_lower_upper(df_1, "Annual_Income")
# still has some otliers

In [ ]:
plt.figure(figsize = (10,5))
sns.histplot(data = df_1, x = "Annual_Income",kde=True)
plt.show()

Monthly_Inhand_Salary

In [ ]:
#Before
df_1.groupby("Customer_ID")["Monthly_Inhand_Salary"].apply(list)

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Monthly_Inhand_Salary")
plt.title("Box Plot of Monthly_Inhand_Salary")
plt.show()

In [ ]:
df_1["Monthly_Inhand_Salary"] = replace_group_outliers_fillna(df_1, "Customer_ID", "Monthly_Inhand_Salary")

In [ ]:
#after
df_1.groupby("Customer_ID")["Monthly_Inhand_Salary"].apply(list)

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Monthly_Inhand_Salary")
plt.title("Box Plot of Monthly_Inhand_Salary")
plt.show()

In [ ]:
get_iqr_lower_upper(df_1, "Monthly_Inhand_Salary")
# still has some otliers

In [ ]:
plt.figure(figsize = (10,5))
sns.histplot(data = df_1, x = "Monthly_Inhand_Salary",kde=True)
plt.show()

In [ ]:
plt.figure(figsize = (10,5))
sns.scatterplot(data=df_1, x="Monthly_Inhand_Salary", y="Annual_Income", hue="Credit_Score")
plt.show()
# high +ve linear corelation between features

In [ ]:
plt.figure(figsize = (10,5))
sns.regplot(data=df_1, x="Monthly_Inhand_Salary", y="Annual_Income")
plt.show()
# high +ve linear corelation between features

Num_Bank_Accounts

In [ ]:
#Before
df_1.groupby("Customer_ID")["Num_Bank_Accounts"].apply(list)

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Num_Bank_Accounts")
plt.title("Box Plot of Num_Bank_Accounts")
plt.show()

In [ ]:
df_1["Num_Bank_Accounts"] = replace_group_outliers_fillna(df_1, "Customer_ID", "Num_Bank_Accounts")

In [ ]:
#after
df_1.groupby("Customer_ID")["Num_Bank_Accounts"].apply(list)

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Num_Bank_Accounts")
plt.title("Box Plot of Num_Bank_Accounts")
plt.show()

In [ ]:
get_iqr_lower_upper(df_1, "Num_Bank_Accounts")

In [ ]:
# plt.figure(figsize = (10,5))
# sns.histplot(data = df_1, x = "Num_Bank_Accounts",kde=True)
# plt.show()

Num_Credit_Card

In [ ]:
#Before
df_1.groupby("Customer_ID")["Num_Credit_Card"].apply(list)

In [ ]:
sns.boxplot(data = df_1, x = "Num_Credit_Card")
plt.title("Box Plot of Num_Credit_Card")
plt.show()

In [ ]:
df_1["Num_Credit_Card"] = replace_group_outliers_fillna(df_1, "Customer_ID", "Num_Credit_Card")

In [ ]:
#after
df_1.groupby("Customer_ID")["Num_Credit_Card"].apply(list)

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Num_Credit_Card")
plt.title("Box Plot of Num_Credit_Card")
plt.show()

In [ ]:
get_iqr_lower_upper(df_1, "Num_Credit_Card")

Interest_Rate

In [ ]:
#Before
df_1.groupby("Customer_ID")["Interest_Rate"].apply(list)

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Interest_Rate")
plt.title("Box Plot of Interest_Rate")
plt.show()

In [ ]:
df_1["Interest_Rate"] = replace_group_outliers_fillna(df_1, "Customer_ID", "Interest_Rate")

In [ ]:
#after
df_1.groupby("Customer_ID")["Interest_Rate"].apply(list)

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Interest_Rate")
plt.title("Box Plot of Interest_Rate")
plt.show()

In [ ]:
get_iqr_lower_upper(df_1, "Interest_Rate")

Num_of_Loan

In [ ]:
#Before
df_1.groupby("Customer_ID")["Num_of_Loan"].apply(list)

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Num_of_Loan")
plt.title("Box Plot of Num_of_Loan")
plt.show()

In [ ]:
df_1["Num_of_Loan"] = replace_group_outliers_fillna(df_1, "Customer_ID", "Num_of_Loan")

In [ ]:
#after
df_1.groupby("Customer_ID")["Num_of_Loan"].apply(list)

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Num_of_Loan")
plt.title("Box Plot of Num_of_Loan")
plt.show()

In [ ]:
get_iqr_lower_upper(df_1, "Num_of_Loan")

Delay_from_due_date

In [ ]:
#Before
df_1.groupby("Customer_ID")["Delay_from_due_date"].apply(list)

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Delay_from_due_date")
plt.title("Box Plot of Delay_from_due_date")
plt.show()

In [ ]:
df_1["Delay_from_due_date"] = replace_group_outliers_fillna(df_1, "Customer_ID", "Delay_from_due_date")

In [ ]:
#after
df_1.groupby("Customer_ID")["Delay_from_due_date"].apply(list)

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Delay_from_due_date")
plt.title("Box Plot of Delay_from_due_date")
plt.show()

In [ ]:
get_iqr_lower_upper(df_1, "Delay_from_due_date")

Num_of_Delayed_Payment

In [ ]:
#Before
df_1.groupby("Customer_ID")["Num_of_Delayed_Payment"].apply(list)

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Num_of_Delayed_Payment")
plt.title("Box Plot of Num_of_Delayed_Payment")
plt.show()

In [ ]:
df_1["Num_of_Delayed_Payment"] = replace_group_outliers_fillna(df_1, "Customer_ID", "Num_of_Delayed_Payment")

In [ ]:
#after
df_1.groupby("Customer_ID")["Num_of_Delayed_Payment"].apply(list)

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Num_of_Delayed_Payment")
plt.title("Box Plot of Num_of_Delayed_Payment")
plt.show()

In [ ]:
get_iqr_lower_upper(df_1, "Num_of_Delayed_Payment")

Changed_Credit_Limit

In [ ]:
#Before
df_1.groupby("Customer_ID")["Changed_Credit_Limit"].apply(list)

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Changed_Credit_Limit")
plt.title("Box Plot of Changed_Credit_Limit")
plt.show()


In [ ]:
df_1["Changed_Credit_Limit"] = replace_group_outliers_fillna(df_1, "Customer_ID", "Changed_Credit_Limit")

In [ ]:
#after
df_1.groupby("Customer_ID")["Changed_Credit_Limit"].apply(list)

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Changed_Credit_Limit")
plt.title("Box Plot of Changed_Credit_Limit")
plt.show()


In [ ]:
get_iqr_lower_upper(df_1, "Changed_Credit_Limit")

Num_Credit_Inquiries

In [ ]:

#Before
df_1.groupby("Customer_ID")["Num_Credit_Inquiries"].apply(list)

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Num_Credit_Inquiries")
plt.title("Box Plot of Num_Credit_Inquiries")
plt.show()


In [ ]:
df_1["Num_Credit_Inquiries"] = replace_group_outliers_fillna(df_1, "Customer_ID", "Num_Credit_Inquiries")

In [ ]:
#after
df_1.groupby("Customer_ID")["Num_Credit_Inquiries"].apply(list)

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Num_Credit_Inquiries")
plt.title("Box Plot of Num_Credit_Inquiries")
plt.show()


Outstanding_Debt

In [ ]:

#Before
df_1.groupby("Customer_ID")["Outstanding_Debt"].apply(list)

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Outstanding_Debt")
plt.title("Box Plot of Outstanding_Debt")
plt.show()

In [ ]:
df_1["Outstanding_Debt"] = replace_group_outliers_fillna(df_1, "Customer_ID", "Outstanding_Debt")

In [ ]:
#after
df_1.groupby("Customer_ID")["Outstanding_Debt"].apply(list)

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Outstanding_Debt")
plt.title("Box Plot of Outstanding_Debt")
plt.show()

Credit_Utilization_Ratio

In [ ]:

#Before
df_1.groupby("Customer_ID")["Credit_Utilization_Ratio"].apply(list)

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Credit_Utilization_Ratio")
plt.title("Box Plot of Credit_Utilization_Ratio")
plt.show()

In [ ]:
df_1["Credit_Utilization_Ratio"].isnull().sum()

In [ ]:
# df_1["Credit_Utilization_Ratio"] = replace_group_outliers_fillna(df_1, "Customer_ID", "Credit_Utilization_Ratio")

In [ ]:
#after
# df_1.groupby("Customer_ID")["Credit_Utilization_Ratio"].apply(list)

Credit_History_Age

In [ ]:
df_1["Credit_History_Age"].isnull().sum()

In [ ]:

#Before
df_1.groupby("Customer_ID")["Credit_History_Age"].apply(list)

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Credit_History_Age")
plt.title("Box Plot of Credit_History_Age")
plt.show()

In [ ]:
df_1['Credit_History_Age'] = df_1.groupby("Customer_ID")["Credit_History_Age"].apply(lambda x: x.interpolate().bfill().ffill())
# filling NaN with interpolate medthod
# bfill > back fill
# ffill > forward fill

In [ ]:
#after
df_1.groupby("Customer_ID")["Credit_History_Age"].apply(list)

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Credit_History_Age")
plt.title("Box Plot of Credit_History_Age")
plt.show()

In [ ]:
lst22 = [182.0, 182.0, 184.0, np.NaN, 186.0]
lst22 = pd.Series(lst22)
lst22

In [ ]:
lst22.mode()
# if i used the replace_group_outliers_fillna function. this mode will replace the NaN
# but instead interpolate see below >> 185 is fill isplace of NaN instead of 182

In [ ]:
lst22.interpolate().bfill().ffill()

Total_EMI_per_month

In [ ]:

#Before
df_1.groupby("Customer_ID")["Total_EMI_per_month"].apply(list)

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Total_EMI_per_month")
plt.title("Box Plot of Total_EMI_per_month")
plt.show()

In [ ]:
df_1["Total_EMI_per_month"] = replace_group_outliers_fillna(df_1, "Customer_ID", "Total_EMI_per_month")

In [ ]:
#after
df_1.groupby("Customer_ID")["Total_EMI_per_month"].apply(list)

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Total_EMI_per_month")
plt.title("Box Plot of Total_EMI_per_month")
plt.show()

In [ ]:
get_iqr_lower_upper(df_1, "Total_EMI_per_month")

Amount_invested_monthly

In [ ]:

#Before
df_1.groupby("Customer_ID")["Amount_invested_monthly"].apply(list)

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Amount_invested_monthly")
plt.title("Box Plot of Amount_invested_monthly")
plt.show()

In [ ]:
df_1["Amount_invested_monthly"] = replace_group_outliers_fillna(df_1, "Customer_ID", "Amount_invested_monthly")

In [ ]:
#after
df_1.groupby("Customer_ID")["Amount_invested_monthly"].apply(list)

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Amount_invested_monthly")
plt.title("Box Plot of Amount_invested_monthly")
plt.show()

Monthly_Balance

In [ ]:

#Before
df_1.groupby("Customer_ID")["Monthly_Balance"].apply(list)

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Monthly_Balance")
plt.title("Box Plot of Monthly_Balance")
plt.show()

In [ ]:
df_1["Monthly_Balance"] = replace_group_outliers_fillna(df_1, "Customer_ID", "Monthly_Balance")

In [ ]:
#after
df_1.groupby("Customer_ID")["Monthly_Balance"].apply(list)

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Monthly_Balance")
plt.title("Box Plot of Monthly_Balance")
plt.show()

Tii now
1. We have fill NaN values from group mode for Object and Numeric features.
2. We change group min max that is outlier or wrong values from numeric features.

Check before Modeling:

Num_Bank_Accounts,
Delay_from_due_date,
Num_of_Delayed_Payment,
Monthly_Balance have negative strange number
Amount_invested_monthly have positive strange number

In [ ]:
missing_data(df_1)
# no null values !

Num_Bank_Accounts

In [ ]:
df_1[df_1["Num_Bank_Accounts"]<0]

In [ ]:
df_1[df_1["Customer_ID"]==22931]
# see this customer has negative bank account which cant be possible

In [ ]:
# hence replace them with zero
df_1.loc[df_1["Num_Bank_Accounts"]<0, "Num_Bank_Accounts"]

In [ ]:
df_1.loc[df_1["Num_Bank_Accounts"]<0, "Num_Bank_Accounts"]=0

In [ ]:
df_1.loc[df_1["Num_Bank_Accounts"]<0, "Num_Bank_Accounts"]

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Num_Bank_Accounts")
plt.title("Box Plot of Num_Bank_Accounts")
plt.show()

Delay_from_due_date

In [ ]:
df_1[df_1["Delay_from_due_date"]<0]["Delay_from_due_date"]

In [ ]:
df_1[df_1["Customer_ID"]==48234].iloc[:,0:15]["Delay_from_due_date"]

In [ ]:
df_1.loc[df_1["Delay_from_due_date"]<0, "Delay_from_due_date"] = np.NaN

In [ ]:
df_1[df_1["Customer_ID"]==48234].iloc[:,0:15]["Delay_from_due_date"]

In [ ]:
df_1["Delay_from_due_date"] = replace_group_outliers_fillna(df_1, "Customer_ID", "Delay_from_due_date")

In [ ]:
df_1[df_1["Customer_ID"]==48234].iloc[:,0:15]["Delay_from_due_date"]

Num_of_Delayed_Payment

In [ ]:
df_1[df_1["Num_of_Delayed_Payment"]<0]

In [ ]:
df_1[df_1["Customer_ID"]==8625]["Num_of_Delayed_Payment"]

In [ ]:
df_1.loc[df_1["Num_of_Delayed_Payment"]<0, "Num_of_Delayed_Payment"] = np.NaN

In [ ]:
df_1[df_1["Customer_ID"]==8625]["Num_of_Delayed_Payment"]

In [ ]:
df_1["Num_of_Delayed_Payment"] = replace_group_outliers_fillna(df_1, "Customer_ID", "Num_of_Delayed_Payment")

In [ ]:
df_1[df_1["Customer_ID"]==8625]["Num_of_Delayed_Payment"]

Monthly_Balance

In [ ]:
df_1[df_1["Monthly_Balance"]<0]

In [ ]:
df_1[df_1["Customer_ID"]==39045]["Monthly_Balance"]

In [ ]:
df_1.loc[df_1["Monthly_Balance"]<0, "Monthly_Balance"] = np.NaN

In [ ]:
df_1[df_1["Customer_ID"]==39045]["Monthly_Balance"]

In [ ]:
df_1["Monthly_Balance"] = replace_group_outliers_fillna(df_1, "Customer_ID", "Monthly_Balance")

In [ ]:
df_1[df_1["Customer_ID"]==39045]["Monthly_Balance"]

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Monthly_Balance")
plt.title("Box Plot of Monthly_Balance")
plt.show()

Amount_invested_monthly

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Amount_invested_monthly")
plt.title("Box Plot of Amount_invested_monthly")
plt.show()

In [ ]:
df_1.loc[df_1["Amount_invested_monthly"]>=10000, "Amount_invested_monthly"] = np.NaN

In [ ]:
# fill group Mode
df_1["Amount_invested_monthly"] = df_1.groupby("Customer_ID")["Amount_invested_monthly"].transform(lambda x: x.mode()[0] if not x.mode().empty else np.NaN)

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data = df_1, x = "Amount_invested_monthly")
plt.title("Box Plot of Amount_invested_monthly")
plt.show()

In [ ]:
missing_data(df_1)

In [ ]:
get_iqr_lower_upper(df_1, "Amount_invested_monthly")

# -------------------DONE ----------------

In [ ]:
df_1.to_csv("cleaned_train.csv",index=False)

In [ ]:
from IPython.display import FileLink, FileLinks
train_file = FileLink(r"cleaned_train.csv", result_html_prefix="Click here to download: ")

In [ ]:

display(train_file)